# Chapter 8: Dimensionality Reduction

---

## 📖 Rangkuman Chapter 8

Chapter ini membahas **Dimensionality Reduction** (Pengurangan Dimensi), sebuah topik penting dalam Machine Learning. Banyak dataset di dunia nyata memiliki ribuan atau bahkan jutaan fitur (dimensi).

Jumlah fitur yang sangat besar ini tidak hanya membuat pelatihan menjadi sangat lambat, tetapi juga dapat mempersulit pencarian solusi yang baik. Masalah ini dikenal sebagai **"the curse of dimensionality"** (kutukan dimensi).

Untungnya, kita sering dapat mengurangi jumlah fitur secara signifikan. Chapter ini akan membahas "kutukan" tersebut dan dua pendekatan utama untuk mengatasinya (Proyeksi dan Manifold Learning), serta tiga teknik populer: **PCA**, **Kernel PCA**, dan **LLE**.

## 🎯 Topics Covered

| No | Topic | Description |
|----|-------|-------------|
| 1 | **The Curse of Dimensionality** | Mengapa data berdimensi tinggi itu aneh dan problematis |
| 2 | **Main Approaches** | Perbedaan antara Proyeksi (Projection) dan Manifold Learning |
| 3 | **PCA (Principal Component Analysis)** | Teknik reduksi dimensi paling populer (linier) |
| 4 | **Explained Variance** | Cara mengukur seberapa banyak informasi yang dipertahankan oleh PCA |
| 5 | **PCA Variants** | Randomized PCA (cepat) dan Incremental PCA (skala besar) |
| 6 | **Kernel PCA (kPCA)** | PCA versi nonlinier menggunakan "kernel trick" |
| 7 | **LLE (Locally Linear Embedding)** | Teknik Manifold Learning untuk "membuka" data yang terpilin |
| 8 | **Other Techniques** | Pengenalan singkat MDS, Isomap, t-SNE, dan LDA |

## 🌌 The Curse of Dimensionality

Kita terbiasa hidup di ruang 3D, sehingga intuisi kita gagal saat membayangkan ruang berdimensi tinggi.

**Masalah Utama:**
* **Data Menjadi Jarang (Sparse):** Di ruang berdimensi tinggi, "ruang" sangatlah besar. Dua titik acak dalam sebuah *unit square* (1x1) rata-rata berjarak 0.52, namun di *unit hypercube* 1.000.000 dimensi, jarak rata-ratanya sekitar 408.25!
* **Semua Titik Ada di "Tepi":** Di hypercube 10.000 dimensi, probabilitas sebuah titik acak berada "sangat dekat dengan batas" adalah > 99.999999%.
* **Risiko Overfitting:** Karena data sangat jarang, instance baru kemungkinan besar akan jauh dari instance pelatihan mana pun. Prediksi menjadi tidak dapat diandalkan (ekstrapolasi besar), yang meningkatkan risiko overfitting.
* **Tidak Dapat Diatasi dengan Data:** Jumlah data yang diperlukan untuk menjaga kepadatan (density) data tumbuh secara **eksponensial** dengan jumlah dimensi.

## 💡 Main Approaches

Ada dua pendekatan utama untuk mengurangi dimensi:

| Approach | Description | Analogi | Kapan Digunakan |
|---|---|---|---|
| **Projection** | Memproyeksikan data ke *subspace* berdimensi lebih rendah (hyperplane). | Membuat bayangan 2D dari objek 3D di atas lantai. | Ketika data "tergeletak" di dekat subspace yang "datar" (linier). |
| **Manifold Learning** | Mengasumsikan data terletak pada *manifold* (bentuk) berdimensi rendah yang terpilin atau tertekuk di dalam ruang berdimensi tinggi. | "Membuka" gulungan Swiss roll (kue gulung) untuk menjadikannya lembaran 2D datar. | Ketika data memiliki struktur nonlinier yang terpilin (seperti Swiss roll). |

## ⚙️ PCA (Principal Component Analysis)

**PCA** adalah algoritme reduksi dimensi yang paling populer. PCA bekerja dengan cara **Proyeksi**.

### Core Idea: Preserving the Variance

Bagaimana PCA memilih hyperplane (subspace) terbaik untuk proyeksi?
Ia memilih hyperplane yang **mempertahankan jumlah variance (varians) semaksimal mungkin**.


Ini setara dengan memilih sumbu yang **meminimalkan mean squared distance** (jarak kuadrat rata-rata) antara data asli dan proyeksinya (ini disebut *reconstruction error*).

### Principal Components (PCs)

PCA menemukan sumbu-sumbu yang memaksimalkan varians:
* **1st PC**: Sumbu yang menyimpan varians terbesar.
* **2nd PC**: Sumbu kedua, yang **ortogonal (tegak lurus)** terhadap PC pertama, yang menyimpan varians tersisa terbesar.
* Dan seterusnya.

### Training PCA

PCA menggunakan **Singular Value Decomposition (SVD)** untuk menemukan PCs.

Di Scikit-Learn, cukup gunakan `PCA()` dan set `n_components` ke jumlah dimensi target (atau rasio varians yang ingin dipertahankan).

In [ ]:
from sklearn.decomposition import PCA
from sklearn.datasets import load_iris

# Load data
iris = load_iris()
X = iris.data

# PCA ke 2 dimensi
pca = PCA(n_components=2)
X2D = pca.fit_transform(X)

print("Shape asli:", X.shape)
print("Shape setelah PCA:", X2D.shape)

## Explained Variance

Atribut `explained_variance_ratio_` memberi tahu seberapa banyak varians yang dipertahankan oleh masing-masing PC.

Contoh pada Iris: PC pertama menyimpan 95.80% varians, PC kedua 3.36% (total 99.16%).

Untuk memilih jumlah dimensi: Set `n_components` ke angka antara 0.0 dan 1.0 (rasio varians yang ingin dipertahankan, misalnya 0.95).

In [ ]:
print("Explained variance ratio:", pca.explained_variance_ratio_)

## PCA Variants

* **Randomized PCA**: Jika `n_components < 0.8 * n_features`, PCA menggunakan versi *randomized* yang lebih cepat.
* **Incremental PCA (IPCA)**: Untuk dataset besar yang tidak muat di memori (out-of-core). Gunakan `partial_fit()` pada mini-batches.
* **Kernel PCA (kPCA)**: Untuk data nonlinier (selanjutnya).

In [ ]:
from sklearn.decomposition import IncrementalPCA
import numpy as np

# IPCA untuk dataset besar
n_batches = 100
inc_pca = IncrementalPCA(n_components=2)
for X_batch in np.array_split(X, n_batches):
    inc_pca.partial_fit(X_batch)

X_reduced = inc_pca.transform(X)

## Kernel PCA (kPCA)

Kernel PCA adalah PCA versi nonlinier, menggunakan **kernel trick** (seperti SVM).

* Bagus untuk data nonlinier (misalnya, Swiss roll).
* Kernel populer: 'linear', 'rbf', 'sigmoid'.
* **Hyperparameter Tuning**: Gunakan GridSearchCV dengan classifier downstream untuk memilih kernel dan hyperparameternya.
* **Rekonstruksi Pre-image**: Anda bisa membalikkan reduksi dengan `inverse_transform()` (meskipun lossy).

In [ ]:
from sklearn.decomposition import KernelPCA

rbf_pca = KernelPCA(n_components=2, kernel="rbf", gamma=0.04)
X_reduced = rbf_pca.fit_transform(X)

## LLE (Locally Linear Embedding)

LLE adalah teknik **Manifold Learning** nonlinier.

* **Cara Kerja**: Mengukur relasi linier lokal antar instance, lalu mencari representasi berdimensi rendah yang mempertahankan relasi itu semaksimal mungkin.
* Bagus untuk membuka manifold yang terpilin (misalnya, Swiss roll).
* **Complexity**: Pelatihan O(d m^2) untuk mencari k-neighbors, O(d n^3) untuk optimasi. Prediksi O(m).

In [ ]:
from sklearn.manifold import LocallyLinearEmbedding

lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10)
X_reduced = lle.fit_transform(X)

## Other Techniques

* **MDS (Multidimensional Scaling)**: Mirip PCA, tapi meminimalkan jarak antar instance (bukan varians).
* **Isomap**: MDS versi graph-based (menghubungkan neighbors).
* **t-SNE**: Untuk visualisasi (2D/3D), menjaga instance mirip tetap dekat.
* **LDA (Linear Discriminant Analysis)**: Klasifikasi linier yang juga reduksi dimensi (proyeksi yang memisahkan kelas).

## 🎯 Kesimpulan Chapter 8

### 📊 Summary Table

In [ ]:
print("="*80)
print("CHAPTER 8 SUMMARY - DIMENSIONALITY REDUCTION")
print("="*80)

summary_table = """
┌──────────────────────┬──────────────────────┬────────────────────────────┐
│ Technique            │ Approach             │ Best For                   │
├──────────────────────┼──────────────────────┼────────────────────────────┤
│ PCA                  │ Projection (Linear)  │ General purpose            │
│                      │                      │ Preserving variance        │
├──────────────────────┼──────────────────────┼────────────────────────────┤
│ Randomized PCA       │ Projection           │ Fast for large n_samples   │
├──────────────────────┼──────────────────────┼────────────────────────────┤
│ Incremental PCA      │ Projection           │ Out-of-core / streaming    │
├──────────────────────┼──────────────────────┼────────────────────────────┤
│ Kernel PCA           │ Projection (Nonlin)  │ Nonlinear data             │
│                      │ Kernel trick         │ Tunable kernels            │
├──────────────────────┼──────────────────────┼────────────────────────────┤
│ LLE                  │ Manifold Learning    │ Unrolling manifolds        │
│                      │ (Nonlinear)          │ Local linear relations     │
├──────────────────────┼──────────────────────┼────────────────────────────┤
│ MDS                  │ Distance Preserv.    │ Similar to PCA             │
├──────────────────────┼──────────────────────┼────────────────────────────┤
│ Isomap               │ Graph-based MDS      │ Geodesic distances         │
├──────────────────────┼──────────────────────┼────────────────────────────┤
│ t-SNE                │ Local Similarities   │ Visualization (2D/3D)      │
├──────────────────────┼──────────────────────┼────────────────────────────┤
│ LDA                  │ Supervised Proj.     │ Classification + Dim Red.  │
└──────────────────────┴──────────────────────┴────────────────────────────┘
"""

print(summary_table)

### 💡 Key Takeaways

**1. Curse of Dimensionality:**
- High-D data: Sparse, far apart, overfitting risk.
- Solution: Reduce dimensions first.

**2. Approaches:**
- Projection: Linear (PCA).
- Manifold: Nonlinear (LLE).

**3. PCA:**
- Preserve variance.
- Variants for large/streaming data.
- Compression (lossy).

**4. kPCA:**
- Nonlinear PCA via kernels.
- Tune with GridSearch.

**5. LLE:**
- Unrolls manifolds.
- Good for twisted data.

**6. Pro/Con Reduction:**
- **Pro:** Faster training, less memory, sometimes better perf.
- **Con:** Lossy, complex pipeline.

## 🔧 Exercises (from the book)

### Exercise 1
**Q:** What are the main motivations for reducing a dataset's dimensionality? What are the main drawbacks?
**A:**
**Motivasi Utama:**
- Speed up training.
- Save memory/storage.
- Visualize data (2D/3D).
- Improve performance (remove noise/useless features).

**Kelemahan:**
- Kehilangan informasi (lossy).
- Dapat menurunkan performa model.
- Membuat pipeline model menjadi lebih kompleks dan lebih sulit dirawat.

---

### Exercise 2
**Q:** What is the curse of dimensionality?
**A:** Ini adalah fakta bahwa banyak hal berperilaku sangat berbeda di ruang berdimensi tinggi. Data menjadi sangat **jarang (sparse)**; instance pelatihan cenderung sangat **jauh satu sama lain**. Ini membuat prediksi menjadi tidak dapat diandalkan (berdasarkan ekstrapolasi besar) dan meningkatkan risiko **overfitting**.

---

### Exercise 3
**Q:** Once a dataset's dimensionality has been reduced, is it possible to reverse the operation?
**A:** **Tergantung.**
* **PCA:** Ya, dapat dibalikkan menggunakan metode `inverse_transform()`. Namun, ini adalah "rekonstruksi" yang **lossy** (ada kehilangan data); Anda tidak mendapatkan data asli kembali karena Anda membuang varians.
* **kPCA:** Ya, tetapi lebih rumit. Ini disebut menemukan "pre-image". Anda dapat mengaktifkannya dengan `fit_inverse_transform=True`.
* **LLE:** (Teks tidak menyebutkan metode inversi langsung).

---

### Exercise 4
**Q:** Can PCA be used to reduce the dimensionality of a highly nonlinear dataset?
**A:** **Ya, bisa,** tapi PCA adalah algoritme linier yang akan mencoba "meratakan" data. Ini akan efektif dalam mengurangi dimensi tetapi akan kehilangan semua struktur nonlinier yang kompleks (misalnya, "menghancurkan" Swiss roll menjadi satu bidang datar). Untuk dataset nonlinier, **kPCA** atau **LLE** adalah pilihan yang jauh lebih baik.

---

### Exercise 5
**Q:** Suppose you perform PCA on a 1,000-dimensional dataset, setting the explained variance ratio to 95%. How many dimensions will the resulting dataset have?
**A:** **Tidak mungkin diketahui** tanpa melihat datasetnya. Jumlah dimensi akan bergantung pada struktur internal data tersebut. Jika datasetnya MNIST (dari 784D), hasilnya adalah 154D. Jika dataset lain, bisa jadi 100D, atau 500D. Jawabannya **bergantung pada dataset**.

---

### Exercise 6
**Q:** In what cases would you use vanilla PCA, Incremental PCA, Randomized PCA, or Kernel PCA?
**A:**
* **Vanilla PCA (Full SVD):** Pilihan default untuk dataset yang muat di memori.
* **Incremental PCA (IPCA):** Ketika dataset **terlalu besar** untuk muat di memori (out-of-core) atau untuk **data streaming (online)**.
* **Randomized PCA:** Ketika $d$ (dimensi target) jauh lebih kecil dari $n$ (fitur asli) dan Anda membutuhkan **performa yang jauh lebih cepat**.
* **Kernel PCA (kPCA):** Ketika dataset **nonlinier** (misalnya, terpilin seperti Swiss roll).

---

### Exercise 7
**Q:** How can you evaluate the performance of a dimensionality reduction algorithm on your dataset?
**A:**
1.  **Kualitas Pipeline (Supervised):** Ukur performa dari *tugas downstream* (misalnya, klasifikasi). Reduksi dimensi yang baik adalah yang meningkatkan akurasi atau sangat mempercepat pelatihan tanpa terlalu merusak akurasi akhir.
2.  **Reconstruction Error (Unsupervised):** Ukur "reconstruction error" (atau "pre-image error" untuk kPCA). Reduksi dimensi yang baik memiliki error rekonstruksi yang rendah.

---

### Exercise 8
**Q:** Does it make any sense to chain two different dimensionality reduction algorithms?
**A:** **Ya.** (Teks tidak secara eksplisit membahas ini, tetapi ini adalah praktik umum). Misalnya, Anda dapat menggunakan PCA untuk menghilangkan redundansi linier dengan cepat, lalu menggunakan t-SNE atau LLE pada data yang sudah dikurangi tersebut untuk memvisualisasikan struktur manifold nonliniernya.

---

### Exercise 9 & 10
**Q:** (MNIST Exercises)
**A:** (Requires implementation)
* **Ex. 9:** Latih Random Forest pada MNIST. Lalu gunakan PCA (95% variance). Latih Random Forest lagi pada data tereduksi. Bandingkan waktu dan akurasi.
* **Ex. 10:** Gunakan t-SNE untuk mengurangi MNIST menjadi 2D dan plot hasilnya. Bandingkan visualisasi dengan PCA, LLE, atau MDS.

---

**Happy Reducing! 📉**